In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

In [2]:
data_dir = "../../datasets/train"
batch_size = 32
epochs = 8
workers = 8

In [3]:
device = torch.device('cuda:0')

In [4]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [5]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + "_cropped"))
    for p, _ in dataset.samples
]

In [6]:
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print("\rBatch {} of {}".format(i+1, len(loader)), end='')

Batch 6 of 6

In [7]:
del(mtcnn)

In [8]:
len(dataset.class_to_idx)

2

In [9]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

In [10]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer=optimizer, milestones=[5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

In [11]:
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8*len(img_inds))]
val_inds = img_inds[int(0.8*len(img_inds)):]

train_inds.shape, val_inds.shape

((152,), (38,))

In [12]:
train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)

val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

In [13]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [14]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' *10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True,
    device=device, writer=writer
)



Initial
----------
Valid |     2/2    | loss:    0.7244 | fps:   88.7120 | acc:    0.4896   


(tensor(0.7244), {'fps': tensor(88.7120), 'acc': tensor(0.4896)})

In [15]:
for epoch in range(epochs):
    print(f'\nEpoch {epoch+1}/{epochs}')
    print('-'*10)
    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )
    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True,
        device=device, writer=writer
    )
writer.close()


Epoch 1/8
----------
Train |     5/5    | loss:    0.1902 | fps:  157.6107 | acc:    0.8792   
Valid |     2/2    | loss:   20.9496 | fps:   91.9187 | acc:    0.3385   

Epoch 2/8
----------
Train |     5/5    | loss:    0.0107 | fps:  119.7791 | acc:    1.0000   
Valid |     2/2    | loss:  127.7221 | fps:   94.8000 | acc:    0.3385   

Epoch 3/8
----------
Train |     5/5    | loss:    0.0100 | fps:  124.9404 | acc:    0.9937   
Valid |     2/2    | loss:    0.0000 | fps:   92.5174 | acc:    1.0000   

Epoch 4/8
----------
Train |     5/5    | loss:    0.0008 | fps:  123.5540 | acc:    1.0000   
Valid |     2/2    | loss:    0.0000 | fps:   92.1170 | acc:    1.0000   

Epoch 5/8
----------
Train |     5/5    | loss:    0.0000 | fps:  122.8081 | acc:    1.0000   
Valid |     2/2    | loss:    0.0000 | fps:   89.4230 | acc:    1.0000   

Epoch 6/8
----------
Train |     5/5    | loss:    0.0004 | fps:  119.9467 | acc:    1.0000   
Valid |     2/2    | loss:    0.0000 | fps:   91.0662 

In [16]:
save_path = '../../models/facenet_v1.pth'

os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(resnet.state_dict(), save_path)

print(f"\nModel successfully saved to {save_path}")


Model successfully saved to ../../models/facenet_v1.pth
